In [63]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [12]:
import nltk
nltk.download('brown')
nltk.download('universal_tagset')
from nltk.corpus import brown

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\VARTUL\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\VARTUL\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [13]:
ls = list(brown.tagged_sents(tagset = 'universal'))


In [14]:
words = list(brown.words())

In [15]:
len(words)

1161192

In [16]:
#add start and end tags:
def add_start_end_tag(sentences_original):
    sentences=[""]*len(sentences_original)
    for i in range(len(sentences_original)):
        sentences[i]=[("<start>","<start>")]+sentences_original[i]+[("<end>","<end>")]
    return sentences

In [17]:
#given a tag, what is its index : tags dict {"tag" : "tag_index"}
#given an index what is the tag : tags_reverse list of string
#count of each tag: tag_count
def creat_tags_meta(sentences):
    tags_dict={}
    tags_reverse=[""]*100000
    cnt = 0
    tag_count=[]
    for i in range(len(sentences)):
        for j in sentences[i]:
            if j[1] not in tags_dict:
                tags_dict[j[1]] = cnt
                tags_reverse[cnt]=j[1]
                tag_count.append(0)
                cnt += 1
            tag_count[tags_dict[j[1]]]+=1

    return (tags_dict, tags_reverse, tag_count)


In [18]:
#creating transition matrix
def create_transition_matrix(sentences, tags_dict, tag_count):
    bigram_matrix = []
    transition_matrix = []
    total_tags = len(tag_count)
    for i in range(total_tags): 
        bigram_matrix.append([0]*total_tags)
        transition_matrix.append([0]*total_tags)

    #creating a bigram matrix
    for sentence in sentences:
        for j in range(len(sentence)-1):
            bigram_matrix[tags_dict[sentence[j][1]]][tags_dict[sentence[j+1][1]]]+=1
    # print(bigram_matrix)
    
    #create transition matrix A
    for i in range(total_tags):
        for j in range(total_tags):
            transition_matrix[i][j]=bigram_matrix[i][j]/tag_count[i]
    # print(transition_matrix)
    
    return (transition_matrix, bigram_matrix)


In [19]:
#word to index number
def create_word_dict(sentences):

    idx = 0
    word_dict={}
    # word_dict["<start>"]=idx
    for sentence in sentences:
        for tup in sentence:
            if tup[0] not in word_dict:
                
                word_dict[tup[0]]=idx
                idx += 1
    word_dict["<unknown>"]=idx
    # print('idx--------------------------------', idx)
    # print('worddict--------------------------------', len(word_dict))
    return word_dict
    

In [20]:
def create_emmision_matrix(word_dict, tags_dict, tag_count, sentences):
    # words = list(brown.words())
    words_cnt = len(word_dict)
    total_tags = len(tags_dict)
    emmision_matrix = []
    
    for i in range(words_cnt):
        emmision_matrix.append([0]*total_tags)
    # print("--------------------------------------",len(word_dict))
    # print("------------------------", len(emmision_matrix))
    # print("------------------------", word_dict['<unknown>'])
    
    for sentence in sentences:
        for j in sentence:
            emmision_matrix[word_dict[j[0]]][tags_dict[j[1]]]+=1

    
    for i in range(words_cnt):
        for j in range(total_tags):
            emmision_matrix[i][j]/=tag_count[j]
    # print("------------------------", emmision_matrix[word_dict['unknown'])

    # handling missing case
    aux=[0]*len(tags_dict.keys())
    aux[tags_dict['NOUN']]=0.5
    aux[tags_dict['VERB']]=0.25
    aux[tags_dict['ADV']]=0.125
    aux[tags_dict['ADJ']]=0.125
    # print(aux)
    emmision_matrix[word_dict['<unknown>']]=aux
    

    return emmision_matrix

In [21]:
def remove_tags(sentence):
    new_sentence=[]
    for i in sentence:
        new_sentence.append(i[0])

    # print(new_sentence)

    return new_sentence

In [60]:
def HMM_logic(input, transition_matrix, emmision_matrix, tags_dict, word_dict):
    print(tags_dict)
    tags_output=["<start>"]
    last_prob = 1
    viterbi=[]
    for i in range(len(tags_dict)):
        viterbi.append(([0])*len(input))
    
    viterbi[tags_dict["<start>"]][0]=1
    
    for i in range(1, len(input)):
        word_prob=0
        tags=""
        curr_ob = input[i]
        if curr_ob not in word_dict:
            curr_ob="<unknown>"
        for curr_tag in tags_dict:
            max_prob=0
            for prev_tag in tags_dict:
                 temp = transition_matrix[tags_dict[prev_tag]][tags_dict[curr_tag]]*emmision_matrix[word_dict[curr_ob]][tags_dict[curr_tag]]*viterbi[tags_dict[prev_tag]][i-1]
                 if temp>max_prob:
                     max_prob=temp
            viterbi[tags_dict[curr_tag]][i]=max_prob
            if max_prob>word_prob:
                word_prob=max_prob
                tags=curr_tag
        if tags=='':
            tags='X'
        tags_output.append(tags)
    

        
        



    return tags_output  

In [23]:
sentences_original = brown.tagged_sents(tagset='universal')
sentences = add_start_end_tag(sentences_original)
# (tags_dict, tags_reverse, tag_count) = creat_tags_meta(sentences)
# (transition_matrix, bigram_matrix) = create_transition_matrix(sentences, tags_dict, tag_count)
# word_dict = create_word_dict(sentences)
# emmision_matrix = create_emmision_matrix(word_dict, tags_dict, tag_count, sentences)

In [24]:
def build_confusion_matrix(cm,actual,predicted,tags_dict):
    for i in range(len(actual)):
        cm[tags_dict[actual[i][1]]][tags_dict[predicted[i]]]+=1

In [25]:
# input=["<start>","do","you","know","who","is","vartul", "?", "<end>"]
# tags_output = HMM_logic(input, transition_matrix, emmision_matrix, tags_dict, word_dict)
# print(tags_output)

In [26]:
#splits data in k sets:
def k_splits (k, data):
    splits = {}
    n = len(data)//k
    for i in range(0, k):
        i = int(i)
        if(i+1 == k):
            splits[i]= data[n*i : ]
        splits[i]=data[n*i : n*(i+1)]
    return splits

In [51]:
cross_validation_set=k_splits(5,sentences)
keys=list(cross_validation_set.keys())

for i in keys:
    train_data=[]
    for k in cross_validation_set.keys():
        if k!=i:
            train_data+=cross_validation_set[k]

    test_data=cross_validation_set[i]
    # print(test_data[1:20])
    # break
    (tags_dict, tags_reverse, tag_count) = creat_tags_meta(train_data)
    confusion_matrix=[]
    for i in range(len(tag_count)):
        confusion_matrix.append([0]*len(tag_count))
    # print(tags_dict)
    (transition_matrix, bigram_matrix) = create_transition_matrix(train_data, tags_dict, tag_count)
    word_dict = create_word_dict(train_data)
    emmision_matrix = create_emmision_matrix(word_dict,  tags_dict, tag_count, train_data)
    # c=0
    for sen in test_data:
        # if c == 10:
        #     break
        # c+=1
        # print(remove_tags(sen))
        tags_output = HMM_logic(remove_tags(sen), transition_matrix, emmision_matrix, tags_dict, word_dict)
        # print(sen,tags_output)
        # print(len(remove_tags(sen)),len(tags_output))
        build_confusion_matrix(confusion_matrix,sen,tags_output,tags_dict)
        
    print(confusion_matrix)

[[11468, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 29294, 3, 10, 14, 0, 34, 34, 183, 7, 0, 1465, 0, 4], [0, 452, 7685, 0, 3, 0, 64, 0, 0, 0, 0, 0, 0, 0], [0, 43, 0, 34765, 2162, 0, 0, 0, 15, 132, 0, 4, 0, 7], [0, 13, 4, 1175, 64553, 1, 13, 0, 35, 851, 0, 28, 83, 34], [0, 0, 0, 0, 0, 30110, 0, 0, 0, 0, 0, 0, 0, 5], [0, 211, 26, 0, 1, 0, 29202, 9, 1, 0, 0, 1, 0, 4], [0, 1, 0, 0, 0, 0, 10, 7633, 17, 0, 0, 0, 0, 1], [0, 496, 1, 24, 190, 0, 50, 29, 9482, 579, 0, 119, 0, 1], [0, 21, 0, 151, 1906, 0, 0, 0, 602, 16223, 0, 32, 4, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11466, 0, 0, 2], [0, 1278, 2, 12, 32, 0, 0, 0, 19, 67, 0, 4380, 0, 1], [0, 0, 0, 11, 445, 0, 0, 0, 0, 0, 0, 0, 3517, 1], [0, 0, 0, 1, 200, 1, 3, 0, 0, 3, 0, 0, 0, 73]]
[[11468, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 30171, 5, 0, 0, 152, 0, 0, 0, 3, 0, 46, 0, 7], [0, 21, 58710, 712, 1020, 9, 0, 0, 51, 0, 4, 2, 107, 47], [0, 0, 1950, 16946, 126, 13, 0, 0, 586, 0, 56, 1, 0, 6], [0, 0, 2699, 138, 35104, 30, 0, 0, 13, 0, 2, 0, 0, 14

In [52]:
actual=0
count=0
for i in range(0,len(confusion_matrix)):
    actual+=confusion_matrix[i][i]
    count+=sum(confusion_matrix[i])
actual/=5
count/=5
print(actual/count)

0.9486702986081903


In [53]:
def recall(confusion_matrix):
    recall = []
    n = len(confusion_matrix)
    for i in range(0,n):
        sum = 0
        for j in range(0,len(confusion_matrix[i])):
            sum+=confusion_matrix[i][j]
        recall.append(confusion_matrix[i][i]/sum)

    return recall

In [54]:
def precision(confusion_matrix):
    precision = []
    n = len(confusion_matrix)
    for i in range(0,n):
        sum = 0
        for j in range(0,len(confusion_matrix[i])):
            sum+=confusion_matrix[j][i]
        precision.append(confusion_matrix[i][i]/sum)

    return precision

In [55]:
print("recall : ", recall(confusion_matrix))

recall :  [1.0, 0.9816774837743385, 0.9583308845136644, 0.8561136478944699, 0.9488402341209625, 0.9049039817728111, 0.9998968469552659, 0.9997384025113359, 0.834449946596757, 0.9958151991965183, 0.8212805158912944, 0.9409460243760882, 0.9850746268656716, 0.1270718232044199]


In [56]:
print("precision : ", precision(confusion_matrix))

precision :  [1.0, 0.9652447106180354, 0.9147056348694359, 0.8736667702184944, 0.9718028419182948, 0.896635132229628, 1.0, 1.0, 0.9184567703323715, 0.9867308011278819, 0.7631616493080325, 0.9986141053279951, 0.8734491315136477, 0.359375]


In [59]:
input=["<start>", "Although", "it", "was", "raining", "heavily", "outside", "the", "children", "decided", "to", "continue", "their", "game", "of", "soccer", "in", "the", "park", "until", "it", "became", "too", "dark", "to", "see", ".", "<end>"]
tags_output= HMM_logic(input, transition_matrix, emmision_matrix, tags_dict, word_dict)
print(tags_output)

{'<start>': 0, 'DET': 1, 'NOUN': 2, 'ADJ': 3, 'VERB': 4, 'ADP': 5, '.': 6, '<end>': 7, 'ADV': 8, 'CONJ': 9, 'PRT': 10, 'PRON': 11, 'NUM': 12, 'X': 13}
['<start>', 'ADP', 'PRON', 'VERB', 'VERB', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PRON', 'VERB', 'ADV', 'ADJ', 'PRT', 'VERB', '.', '<end>']
